In [ ]:
!pip install TTS
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers
!pip install pygame

In [ ]:
import time
from TTS.api import TTS
import os
import datetime
import logging
import torch
from pygame import mixer
import torch
from transformers import pipeline
from textwrap import dedent
# Initialize the TTS engine
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2")
tts.to(device)
pipe = None

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

# Initialize the mixer for audio playback
mixer.init()

def load_model():
    
    pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")
    return pipe

def generate_response(text):
    """Generate a response to the user's messages."""

    messages = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who is proficient in Science and Space. You can answer questions and provide information on these topics.",
        },
        {"role": "user", "content": text},
    ]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=1024, do_sample=True, temperature=0.1, top_k=50, top_p=0.95)
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    output_file = f"output_{timestamp}.wav"

    with open("text_gen_output.txt", 'w', encoding='utf-8') as file:
        file.write(outputs[0]["generated_text"])
    unparsed_response = outputs[0]["generated_text"]
    parsed_response = unparsed_response.split("<|assistant|>\n")[-1]
    print(f"Response: {parsed_response}")
    try:
        tts.tts_to_file(text=parsed_response, file_path=output_file,speaker_wav="samples_en_sample.wav", language="en")
        logging.info(f"Processed text to {output_file}")
        # Play the output file
        mixer.music.load(output_file)
        mixer.music.play()
        while mixer.music.get_busy():  # wait for the audio to finish playing
            time.sleep(1)
    except Exception as e:
        logging.error(f"Error processing text: {str(e)}")
    return outputs[0]["generated_text"]

if __name__ == "__main__":
    pipe = load_model()
    generate_response(dedent("""Please answer the clue's below please.
Detected patterns in 'sci.med' from the newsgroup dataset:

Clue: the happy chemical in your brain i influence mood both joy and pain
Answer:

Clue: from tryptophan my form takes flight a neurotransmitter shining bright
Answer:

Clue: tiny creatures unseen by the eye i exist in numbers that reach for the sky
Answer:

Clue: prokaryotic cells simple yet grand i shape the world on water and land
Answer:

Clue: within your belly a bustling scene i aid digestion a microbiome team
Answer:


Detected patterns in 'sci.space' newsgroup dataset clues:
Clue: to break earths grasp and touch the sky where rockets roar and dreams take flight so high
Answer:

Clue: defying gravitys relentless hold a journey embarked a story to unfold among the stars new frontiers to see the boundless quest of humanity
Answer:

Clue: a celestial eye orbiting bright beaming down signals day and through night
Answer:

Clue: earths silent companion a technological feat spinning its dance in a rhythm so sweet whispers of data from distant space it charts the cosmos with unwavering grace
Answer:

"""))